# AI Contact Center Coach V Dec 25

## What this notebook does (end-to-end)

This notebook is a research/education prototype that evaluates a contact-center call transcript for script adherence, resolution promises, and coaching feedback. It produces a scored, section-by-section report plus an AI-generated coaching summary.

### 1) Setup and dependencies
- Loads environment variables (expects `OPENAI_API_KEY`).
- Imports OpenAI client for GPT-4 coaching, `sentence_transformers` for semantic similarity, and standard libraries for parsing and scoring.
- If the API key is missing, execution stops with an error.

### 2) Reference scripts and benchmarks
- Defines a **golden script**: example agent utterances for each call section (Greeting, Issue Identification, Troubleshooting, Solution Delivery, Resolution/Ticketing, Upsell, Closing).
- Defines **benchmark scores** (75th percentile targets) per section, plus overall and CSAT targets.

### 3) Input transcript
- Uses a diarized transcript string (`Agent:` / `Customer:` lines) without explicit section markers.

### 4) Heuristic segmentation
- Segments the transcript into sections using keyword matching **only on agent lines**.
- A `current_section` is set based on the latest matched agent line, and **all subsequent lines** are grouped into that section until another agent line switches it.
- This produces a map from section name to its collected utterances (agent + customer).

### 5) Resolution promise extraction and validation (simulated)
- Extracts agent statements that look like **promises of action** (e.g., open a ticket, send an email, notify the customer).
- Runs a **simulated validation** function that returns a status (e.g., validated or pending). In a real system, this would query actual ticketing/email systems.

### 6) Script adherence scoring per section
- For each section, it compares **only the agent utterances in that section** to the golden script.
- Uses the `all-MiniLM-L6-v2` sentence transformer to compute cosine similarity.
- Each golden sentence is matched to its **best** agent sentence; the section score is the **average** of those best-match similarities.
- Special handling replaces `[agent name]` in golden sentences with a detected name from the agent’s utterance.

### 7) Overall adherence and detailed feedback
- Computes a **weighted overall adherence score** from section scores.
- Generates a detailed markdown report showing per-section adherence vs. benchmarks.

### 8) GPT-4 coaching summary
- Builds a prompt that includes:
  - The full transcript,
  - Section adherence scores,
  - Golden script reference options,
  - Resolution promise validation results,
  - CSAT score.
- Calls GPT-4 to produce a coaching summary with sentiment analysis, strengths, improvement areas, and suggested phrasing.

### 9) Output
- Displays markdown outputs for:
  - Per-section similarity comparisons,
  - Resolution promise checks (highlighting unfulfilled items),
  - Overall and section-wise feedback,
  - GPT-4 coaching narrative.

### Important notes
- This is a **prototype** for research/education, not production.
- Resolution validation is **mocked**; replace with real system integrations for operational use.
- The segmentation is **heuristic** and may misclassify sections if the wording changes.


## Purpose and Use

This notebook is provided for research/education and prototype evaluation only.
It evaluates contact-center calls for script adherence, resolution promises, and coaching feedback.
It analyzes contact-center interactions and may be classified as high-risk AI in some jurisdictions.
Do not use in production without completing required compliance, privacy, and security steps.

## License

MIT License. Copyright (c) 2025 Javier Castro (dnAI). See `LICENSE`.

## Disclaimer

Provided "AS IS" without warranty or liability. Not legal advice.

## Security

Report vulnerabilities to `javiercastro@aiready.es`.
See `.github/SECURITY.md` for the full policy and PGP key.


In [ ]:
# Purpose: research/education prototype only.
# License: MIT (see LICENSE).
# Security: report issues to javiercastro@aiready.es (see .github/SECURITY.md).

# V 3.3
# Developed by Javier Castro, dnAI
# Enhanced to include comprehensive resolution promise patterns and visual highlighting for unfulfilled promises

from openai import OpenAI
import os
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer, util
import numpy as np
import json
import re
import html as html_lib
from IPython.display import Markdown, display, HTML

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Missing OPENAI_API_KEY. Set it in your environment or a local .env file.")

# -------------------------------
# Helper Function for Replacing Agent Name
# -------------------------------
def replace_agent_name(golden_sentence, agent_utterance):
    """
    If the golden sentence contains "[agent name]", try to extract a name from the agent utterance
    and replace the placeholder.
    """
    if "[agent name]" in golden_sentence:
        match = re.search(r'\b([A-Z][a-z]+)\b', agent_utterance)
        if match:
            agent_name = match.group(1)
            return golden_sentence.replace("[agent name]", agent_name)
    return golden_sentence

# -------------------------------
# Golden Scripts for Each Section (Approximately 500 words total)
# -------------------------------
golden_scripts = {
    "Greeting Section": [
        "Hello, thank you for calling ConnectPlus. My name is [agent name]. How may I assist you today? Good morning, and welcome to ConnectPlus support, where we’re dedicated to resolving your issues efficiently. I am [agent name], and I’m here to ensure you have a seamless experience. Whether you need technical support or general assistance, I’m here to help with any inquiries you might have.",
        "Hi, thank you for reaching ConnectPlus. My name is [agent name]. What can I do for you today? Welcome to ConnectPlus customer service – I’m [agent name] and I look forward to helping you get connected and solving any issues you may face."
    ],
    "Issue Identification Section": [
        "I’m sorry to hear you are experiencing difficulties with your service. Could you please describe in detail the issues you’re facing? For instance, are you having trouble connecting or is your connection unstable? I would like to verify your account details to understand the root cause better. Your input is invaluable in helping me address the issue promptly.",
        "I understand you’re encountering problems with your connection. Please explain what you’re experiencing and any error messages you might have seen. Let me also check your account and recent activity so we can pinpoint the problem."
    ],
    "Troubleshooting Section": [
        "Based on our initial check, it appears there is a disruption affecting your area. Let’s perform a few diagnostic tests to pinpoint the issue. Please check if your modem’s indicator lights are steady or blinking; this will help determine whether the signal is stable. We’ll then proceed with some troubleshooting steps such as rebooting your router if necessary.",
        "Our system shows some instability in your neighborhood. To isolate the problem, kindly confirm whether the lights on your modem are flashing or remain constant. We will try a few troubleshooting measures together, and your cooperation is much appreciated."
    ],
    "Solution Delivery Section": [
        "Thank you for your patience. Our analysis indicates that the issue is due to a local network disruption. Our technical team has been alerted and is actively working on restoring normal service. We expect the problem to be resolved within the next two hours, and I will ensure you receive timely updates via text.",
        "I appreciate your cooperation. It appears that a network disruption is affecting your service, and our engineers are addressing it as a priority. We anticipate the issue will be resolved shortly, and you’ll be notified once service is restored."
    ],
    "Resolution (Ticket Creation) Section": [
        "I understand your issue requires further investigation. Let me open a support ticket for your case right away, and I will send you a confirmation email shortly.",
        "Your issue has been logged in our system. A support ticket has been created, and you will receive an email confirmation within a few minutes."
    ],
    "Upsell (AIDA) Section": [
        "In addition to resolving your issue, I’d like to inform you about a special promotion we’re currently running. We are offering an exclusive discount on our Premium Internet plan, which provides faster speeds, enhanced connectivity, and additional features like a complimentary streaming subscription. Would you be interested in receiving more information about this offer?",
        "Furthermore, we have a limited-time offer on our upgraded service package that includes bonus features and faster speeds. If you’re interested, I’d be happy to send you the details so you can take advantage of this promotion."
    ],
    "Closing Section": [
        "Before we conclude, is there anything else I can assist you with today? Thank you for choosing ConnectPlus; it has been a pleasure helping you. Should you have any further questions, please feel free to call us again. Have a wonderful day, and thank you for calling ConnectPlus.",
        "I hope I have addressed all your concerns. Thank you for contacting ConnectPlus. We value your business and are committed to your satisfaction. If you need further assistance in the future, don’t hesitate to reach out. Enjoy your day and stay safe!"
    ]
}

# -------------------------------
# Benchmarks (75th Percentile) for Each Section and Overall
# -------------------------------
benchmarks = {
    "Greeting Section": 80.0,
    "Issue Identification Section": 70.0,
    "Troubleshooting Section": 65.0,
    "Solution Delivery Section": 60.0,
    "Resolution (Ticket Creation) Section": 70.0,
    "Upsell (AIDA) Section": 55.0,
    "Closing Section": 75.0,
    "Overall": 75.0,
    "CSAT": 7.5
}

# -------------------------------
# Diarised Transcription (No explicit section markers)
# -------------------------------
full_transcript = """
Agent: Good afternoon! You’ve reached the support line for ConnectPlus. My name’s Monica. How may I help you today?
Customer: Hi. My connection has been dropping all morning.
Agent: Oh no, that must be frustrating. Let’s get it sorted. Can you describe what exactly is happening?
Customer: Well, it disconnects every few minutes, especially when I’m on video calls.
Agent: Got it. Give me a second to access your account and see if anything unusual pops up… Alright, I’ve pulled up your details.
Agent: I’m checking our systems now. There’s some instability reported in your neighborhood since early morning.
Customer: So it’s a local outage?
Agent: It appears so. But to be sure, let’s do a quick check. Can you confirm whether your modem’s Internet light is blinking or steady?
Customer: It’s blinking.
Agent: Okay, that confirms it. The signal isn’t stable. Just to double-check, can you reboot the router for me?
Customer: Sure. Done.
Agent: Thanks! It’ll take a minute to reconnect… Looks like it’s trying to stabilize now.
Customer: When will it be fixed?
Agent: The engineers estimate about 90 minutes. We’ll notify you via text once everything’s up and running.
Customer: Alright.
Agent: I understand your issue requires further investigation. Let me open a support ticket for your case right away.
Customer: Yes, please open a ticket.
Agent: I understand your issue requires further investigation. Let me open a support ticket for your case right away, and I will send you a confirmation email shortly.
Customer: Thank you.
Agent: While we’re at it, I noticed you’re on the Basic Plan. We’re currently offering 20% off on our Premium plan with double the speed and a streaming service included.
Customer: Oh? Interesting.
Agent: Would you like me to send over the details by email or walk you through it briefly?
Customer: Email’s good.
Agent: Perfect! I’ve sent it to the address we have on file. You can upgrade in just one click if you’re interested.
Customer: Thanks.
Agent: Before we finish, is there anything else I can assist you with today?
Customer: Nope, that’s all.
Agent: Thanks for calling ConnectPlus, [Customer Name]. It was a pleasure helping you. Have a great rest of your day!
Customer: You too. Bye.
"""

# -------------------------------
# Function to Format Golden Scripts for GPT-4 Prompt
# -------------------------------
def format_golden_scripts(scripts):
    """
    Format the golden scripts dictionary into a bullet-pointed markdown string.
    """
    md = "**Golden Script for Each Section:**\n\n"
    for section, sentences in scripts.items():
        md += f"- **{section}:**\n"
        for i, sentence in enumerate(sentences, 1):
            md += f"   - Option {i}: \"{sentence}\"\n"
        md += "\n"
    return md

golden_script_md = format_golden_scripts(golden_scripts)

# -------------------------------
# Heuristic-based Automatic Segmentation Function
# -------------------------------
section_keywords = [
    ("Greeting Section", ["good afternoon", "thank you for calling", "my name", "how may i help"]),
    ("Issue Identification Section", ["frustrating", "describe", "dropping", "issue", "account"]),
    ("Troubleshooting Section", ["checking our systems", "instability", "modem", "blinking", "reboot"]),
    ("Solution Delivery Section", ["engineers estimate", "notify you via text", "fixed", "up and running"]),
    ("Resolution (Ticket Creation) Section", ["open a support ticket", "ticket", "log your issue"]),
    ("Upsell (AIDA) Section", ["basic plan", "20% off", "promotion", "offer", "upgrade"]),
    ("Closing Section", ["before we finish", "thanks for calling", "great rest of your day", "bye", "take care"])
]

def auto_segment_transcript(transcript):
    """
    Automatically segment the diarised transcript into sections based on heuristic keyword matching.
    Returns a dictionary mapping section names to lists of utterances.
    """
    sections = {sec: [] for sec, _ in section_keywords}
    current_section = "Greeting Section"  # Default section
    lines = transcript.splitlines()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        # Check if it's an Agent utterance that triggers a section change
        if line.lower().startswith("agent:"):
            for sec, keywords in section_keywords:
                for keyword in keywords:
                    if keyword in line.lower():
                        current_section = sec
                        break
        sections[current_section].append(line)
    return sections

# -------------------------------
# Extraction Functions
# -------------------------------
def extract_agent_utterances(section_lines):
    """
    Given a list of utterances in a section, extract only those by the agent.
    """
    agent_lines = []
    for line in section_lines:
        if line.startswith("Agent:"):
            agent_lines.append(line[len("Agent:"):].strip())
    return agent_lines

# -------------------------------
# Resolution Extraction Functions (Enriched)
# -------------------------------
def extract_resolution_promises(transcript):
    """
    Extract phrases where the agent makes a promise of action.
    This function looks for key phrases indicating promises such as:
      - "let me open", "I will open", "open a support ticket", "log your issue"
      - "create a ticket", "raise a ticket", "file a ticket", "submit a ticket"
      - "I'll send a confirmation email", "email you", "confirm via email"
      - "update our system", "check our logs", "record your issue", "document your concern"
      - "raise an incident", "initiate a case", "follow up", "escalate", "track your case"
      - "notify", "verify", "process your request", "handle your problem", "resolve your issue"
      - "checking our systems", "sent it to the address"
    """
    promises = []
    pattern = (
        r'\b('
        r'let me open|'
        r'i will open|'
        r'open (a )?(support )?ticket|'
        r'log (your|this) (issue|problem)|'
        r'create (a )?(support )?ticket|'
        r'raise (a )?(support )?ticket|'
        r'file (a )?(support )?ticket|'
        r'submit (a )?(support )?ticket|'
        r'i\'?ll (create|open|log) (a )?(support )?ticket|'
        r'send (a )?confirmation email|'
        r'email( you)? (the )?details|'
        r'confirm( via email)?|'
        r'update (our )?system|'
        r'check (our )?(logs|system)|'
        r'record (your|this) (issue|problem|concern)|'
        r'document (your|this) (issue|problem|concern)|'
        r'raise (an )?incident|'
        r'initiate (a )?case|'
        r'i\'?ll (follow up|escalate|track|notify|verify|update|process|handle|resolve)|'
        r'make sure (to )?(open|create|log) (a )?(support )?ticket|'
        r'checking (our )?systems( now| immediately)?|'
        r'sent( it)? to (the )?address'
        r')\b'
    )
    lines = transcript.splitlines()
    for line in lines:
        if line.lower().startswith("agent:"):
            if re.search(pattern, line, re.IGNORECASE):
                promises.append(line.strip())
    return promises

def check_resolution_action(promise):
    """
    Simulate the validation of a promised action by 'checking' logs or transactions.
    In a real implementation, this function would make API calls or database queries.
    """
    promise_lower = promise.lower()
    if "ticket" in promise_lower or "support ticket" in promise_lower or "log your issue" in promise_lower or "open a support ticket" in promise_lower or "create a support ticket" in promise_lower:
        print("Simulating API call: Checking if a support ticket was created.")
        return {"action": "support ticket creation", "status": "validated", "details": "Support ticket #T123 processed at 15:05."}
    elif "sent" in promise_lower and "address" in promise_lower:
        print("Simulating API call: Checking if a confirmation email was sent.")
        return {"action": "email confirmation", "status": "pending", "details": "Latest email was not sent."}
    elif "checking our systems" in promise_lower:
        print("Simulating API call: Checking system logs for instability.")
        return {"action": "system check", "status": "validated", "details": "Instability in the area confirmed in system logs."}
    else:
        print("Simulating API call: Checking for other types of resolution actions.")
        return {"action": "other", "status": "unknown", "details": "No matching log found."}

# -------------------------------
# Section 1: Script Adherence and Scoring Functions (Direct Average Method)
# -------------------------------
def compute_section_adherence(golden_sentences, agent_sentences, threshold=0.75):
    """
    Compute the adherence percentage for a section by comparing each golden sentence 
    against all agent utterances using cosine similarity, then taking the direct average.
    
    Returns:
      adherence_percentage (float): Average similarity (as a percentage) of the best matches.
      best_match_scores (list): Best cosine similarity score for each golden sentence.
      comparisons (list): For each golden sentence, a dict containing:
            - "golden_sentence": expected sentence.
            - "agent_sentence": best-matching agent utterance.
            - "similarity_score": cosine similarity score.
    """
    model = SentenceTransformer('all-MiniLM-L6-v2')
    agent_embeddings = model.encode(agent_sentences, convert_to_tensor=True)
    best_match_scores = []
    comparisons = []
    for golden_sentence in golden_sentences:
        if "[agent name]" in golden_sentence:
            best_similarity = -1
            best_idx = None
            for j, candidate in enumerate(agent_sentences):
                processed_golden = replace_agent_name(golden_sentence, candidate)
                embedding_processed = model.encode([processed_golden], convert_to_tensor=True)
                similarity = util.cos_sim(embedding_processed, agent_embeddings[j:j+1])
                similarity_value = similarity.item()
                if similarity_value > best_similarity:
                    best_similarity = similarity_value
                    best_idx = j
            best_match_scores.append(best_similarity)
            comparisons.append({
                "golden_sentence": golden_sentence,
                "agent_sentence": agent_sentences[best_idx] if best_idx is not None else "No matching sentence found",
                "similarity_score": best_similarity
            })
        else:
            embedding_golden = model.encode([golden_sentence], convert_to_tensor=True)
            similarities = util.cos_sim(embedding_golden, agent_embeddings)
            best_similarity = similarities.max().item()
            best_idx = int(similarities.argmax())
            best_match_scores.append(best_similarity)
            comparisons.append({
                "golden_sentence": golden_sentence,
                "agent_sentence": agent_sentences[best_idx],
                "similarity_score": best_similarity
            })
    adherence_percentage = np.mean(best_match_scores) * 100
    return adherence_percentage, best_match_scores, comparisons

def compute_overall_adherence(sections_adherence, weights=None):
    """
    Compute overall script adherence as the weighted average of section adherence scores.
    """
    if weights:
        weighted_sum = 0
        total_weight = 0
        for section, adherence in sections_adherence.items():
            weight = weights.get(section, 1)
            weighted_sum += adherence * weight
            total_weight += weight
        overall = weighted_sum / total_weight
    else:
        overall = sum(sections_adherence.values()) / len(sections_adherence)
    return overall

# -------------------------------
# Section 2: GPT-4 Coaching Function (with Golden Script and Resolution Results in Prompt)
# -------------------------------
def get_gpt4_coaching(transcript, section_scores, csat, golden_script_md, resolution_results, openai_api_key):
    """
    Call GPT-4 to generate qualitative coaching feedback, including both the attention and resolution aspects.
    """
    client = OpenAI(api_key=openai_api_key)
    resolution_feedback = "\n".join([f"Promise: \"{r['promise']}\" -> Action: {r['result']['action']}, Status: {r['result']['status']}" 
                                     for r in resolution_results])
    
    prompt = f"""
You are an expert call center coach. Analyze the following full call transcript and the computed script adherence scores.
Below is the golden script for each section:
{golden_script_md}

The transcript is:
\"\"\"{transcript}\"\"\"

The section adherence scores (in %) are:
{json.dumps(section_scores, indent=2)}

The customer satisfaction (CSAT) score for this call is {csat} out of 10.

Additionally, here are the resolution verification results from the call:
{resolution_feedback}

Provide a detailed coaching summary with:
  - A customer sentiment analysis that describes sentiment at the start, middle, and end (or key moments) and notes shifts with brief transcript quotes.
  - What the agent did well in both communication and fulfilling operational commitments.
  - Areas for improvement, especially if any resolution promises (e.g., opening a support ticket or sending a confirmation email) were not validated.
  - Specific suggestions on how to improve low-scoring sections and ensure that promised actions are confirmed.
For each suggestion, include example utterances from the golden script that the agent should consider using, and explain how to check and log operational actions.
Please use bullet points and indentation in your response.
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful call center coaching assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=700
        )
        coaching_feedback = response.choices[0].message.content
    except Exception as e:
        coaching_feedback = f"Error calling GPT-4: {e}"
    return coaching_feedback

# -------------------------------
# Section 3: Detailed Feedback Functions (Using Benchmarks)
# -------------------------------
def generate_section_feedback(section_name, adherence, benchmark):
    """
    Generate detailed feedback for an individual section using the agent's adherence score
    and the benchmark.
    """
    diff = adherence - benchmark
    if diff >= 0:
        feedback = (f"  - Your adherence score is {adherence:.2f}% (benchmark: {benchmark:.2f}%). "
                    f"You are {diff:.2f} points above the target. Great job!")
    else:
        feedback = (f"  - Your adherence score is {adherence:.2f}% (benchmark: {benchmark:.2f}%). "
                    f"You are {abs(diff):.2f} points below the target. Consider using these golden script options:\n"
                    f"      * Example: \"{golden_scripts[section_name][0]}\"\n"
                    f"      * Or: \"{golden_scripts[section_name][1]}\"")
    return feedback

def generate_overall_feedback(overall_adherence, csat, overall_benchmark, csat_benchmark):
    """
    Generate overall feedback using the agent's overall adherence and CSAT scores compared to benchmarks.
    """
    overall_diff = overall_adherence - overall_benchmark
    csat_diff = csat - csat_benchmark
    feedback = ""
    if overall_diff >= 0:
        feedback += f"- **Overall Script Adherence** is {overall_adherence:.2f}% (benchmark: {overall_benchmark:.2f}%). You are {overall_diff:.2f} points above the target.\n"
    else:
        feedback += f"- **Overall Script Adherence** is {overall_adherence:.2f}% (benchmark: {overall_benchmark:.2f}%). You are {abs(overall_diff):.2f} points below the target.\n"
    if csat_diff >= 0:
        feedback += f"- **Customer Satisfaction (CSAT)** is {csat} out of 10 (benchmark: {csat_benchmark}). You are {csat_diff:.2f} points above the target.\n"
    else:
        feedback += f"- **Customer Satisfaction (CSAT)** is {csat} out of 10 (benchmark: {csat_benchmark}). You are {abs(csat_diff):.2f} points below the target.\n"
    return feedback

def generate_detailed_feedback(section_scores, overall_adherence, csat, benchmarks):
    """
    Generate a complete feedback report with overall and per-section explanations,
    including both the agent's score and the benchmark with point differences.
    """
    feedback = {}
    feedback["Overall"] = generate_overall_feedback(overall_adherence, csat, benchmarks["Overall"], benchmarks["CSAT"])
    section_feedback = {}
    for section, adherence in section_scores.items():
        if section in benchmarks:
            section_feedback[section] = generate_section_feedback(section, adherence, benchmarks[section])
        else:
            section_feedback[section] = generate_section_feedback(section, adherence, 75.0)
    feedback["Sections"] = section_feedback
    return feedback

# -------------------------------
# Dashboard UI Builder (HTML)
# -------------------------------

def build_dashboard_html(overall_adherence, csat_score, benchmarks, section_scores, resolution_results, transcript, golden_scripts):
    def esc(text):
        return html_lib.escape(str(text))

    def truncate(text, max_len=110):
        text = str(text)
        return text if len(text) <= max_len else text[: max_len - 3].rstrip() + "..."

    def strip_speaker(line):
        return line.split(":", 1)[1].strip() if ":" in line else line.strip()

    lines = [line.strip() for line in transcript.splitlines() if line.strip()]
    first_customer = next((line for line in lines if line.lower().startswith("customer:")), "")
    last_agent = next((line for line in reversed(lines) if line.lower().startswith("agent:")), "")

    start_quote = truncate(strip_speaker(first_customer), 90)
    end_quote = truncate(strip_speaker(last_agent), 90)

    total_promises = len(resolution_results)
    pending_promises = sum(1 for entry in resolution_results if entry["result"].get("status") == "pending")
    resolution_integrity = (sum(1 for entry in resolution_results if entry["result"].get("status") == "validated") / total_promises * 100) if total_promises else 100.0

    top_section = max(section_scores, key=section_scores.get) if section_scores else "N/A"
    bottom_section = min(section_scores, key=section_scores.get) if section_scores else "N/A"

    section_rows = []
    for section, score in section_scores.items():
        benchmark = benchmarks.get(section, 75.0)
        score_pct = max(0.0, min(100.0, score))
        benchmark_pct = max(0.0, min(100.0, benchmark))
        if benchmark > 0 and score <= 0.8 * benchmark:
            status = "bad"
        elif score < benchmark:
            status = "warn"
        else:
            status = "good"
        section_rows.append(
            f"""
<div class="bar-row">
  <div class="label">{esc(section)}</div>
  <div class="bar-track"><div class="bar actual {status}" style="width: {score_pct:.1f}%"></div></div>
  <div class="bar-track"><div class="bar benchmark" style="width: {benchmark_pct:.1f}%"></div></div>
  <div class="score">{score:.1f}%</div>
</div>
""".strip()
        )

    resolution_items = []
    for entry in resolution_results:
        status = entry["result"].get("status", "unknown")
        label = truncate(strip_speaker(entry["promise"]), 80)
        resolution_items.append(
            f"""<div class="promise"><strong>{esc(status.title())}</strong> — {esc(label)}</div>"""
        )
    if not resolution_items:
        resolution_items.append('<div class="promise">No promises detected.</div>')

    html = f"""
<link rel="preconnect" href="https://fonts.googleapis.com" />
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin />
<link href="https://fonts.googleapis.com/css2?family=Fraunces:wght@600;700&family=Space+Grotesk:wght@400;500;600&display=swap" rel="stylesheet" />
<style>
.dashboard {{
  font-family: 'Space Grotesk', sans-serif;
  background: linear-gradient(140deg, #f8f1e8, #f1efe9 40%, #ecf7f5 100%);
  border: 1px solid #e6d8c7;
  border-radius: 22px;
  padding: 28px;
  box-shadow: 0 24px 60px rgba(30, 29, 27, 0.08);
}}
.dashboard h2 {{
  font-family: 'Fraunces', serif;
  margin: 0 0 12px 0;
}}
.grid {{
  display: grid;
  grid-template-columns: repeat(12, 1fr);
  gap: 16px;
}}
.card {{
  background: #fff7ed;
  border-radius: 18px;
  border: 1px solid #ead9c8;
  padding: 18px;
}}
.metric {{
  display: grid;
  gap: 6px;
}}
.metric strong {{
  font-size: 1.3rem;
}}
.bar-row {{
  display: grid;
  grid-template-columns: 160px 1fr 1fr 60px;
  gap: 10px;
  align-items: center;
  margin-bottom: 10px;
  font-size: 0.85rem;
}}
.bar-track {{
  background: #f3e8db;
  border-radius: 999px;
  height: 10px;
  overflow: hidden;
}}
.bar {{
  height: 100%;
  border-radius: 999px;
}}
.bar.actual.bad {{ background: #e53935; }}
.bar.actual.warn {{ background: #f9a825; }}
.bar.actual.good {{ background: #43a047; }}
.bar.benchmark {{ background: #1e88e5; }}
.promise {{
  padding: 8px 10px;
  background: #fff;
  border-radius: 12px;
  border: 1px solid #ead9c8;
  margin-bottom: 8px;
  font-size: 0.85rem;
}}
.small {{ color: #6f6a64; font-size: 0.85rem; }}
</style>
<div class="dashboard">
  <div class="grid">
    <div class="card" style="grid-column: span 4;">
      <h2>Performance</h2>
      <div class="metric"><span class="small">Overall Adherence</span><strong>{overall_adherence:.1f}%</strong></div>
      <div class="metric"><span class="small">CSAT</span><strong>{csat_score:.1f} / 10</strong></div>
      <div class="metric"><span class="small">Resolution Integrity</span><strong>{resolution_integrity:.1f}%</strong></div>
      <div class="small">Pending promises: {pending_promises}</div>
    </div>
    <div class="card" style="grid-column: span 4;">
      <h2>Highlights</h2>
      <div class="small">Top Section: {esc(top_section)}</div>
      <div class="small">Lowest Section: {esc(bottom_section)}</div>
      <div class="small">Start: “{esc(start_quote)}”</div>
      <div class="small">End: “{esc(end_quote)}”</div>
    </div>
    <div class="card" style="grid-column: span 4;">
      <h2>Promises</h2>
      {''.join(resolution_items)}
    </div>
    <div class="card" style="grid-column: span 12;">
      <h2>Section Adherence vs Benchmark</h2>
      {''.join(section_rows)}
    </div>
  </div>
  <div class="small" style="margin-top: 10px;">Developed by Javier Castro (dnAI)</div>
</div>
"""
    return html

# -------------------------------
# Section 4: Main Function and Execution
# -------------------------------
def main():
    # Automatically segment the diarised transcript into sections using heuristic keywords
    sections = auto_segment_transcript(full_transcript)
    
    # For each section, extract agent utterances and compare with the golden script
    csat_score = 4

    section_scores = {}
    detailed_comparisons = {}
    
    for section_name, golden_lines in golden_scripts.items():
        if section_name in sections:
            section_content = sections[section_name]
            agent_utterances = extract_agent_utterances(section_content)
            adherence, best_scores, comparisons = compute_section_adherence(golden_lines, agent_sentences=agent_utterances)
            section_scores[section_name] = adherence
            detailed_comparisons[section_name] = comparisons
            md_section = f"**--- {section_name} ---**\n\n"
            md_section += f"- **Adherence:** {adherence:.2f}%\n\n"
            for comp in comparisons:
                md_section += f"   - **Golden Sentence:** \"{comp['golden_sentence']}\"\n"
                md_section += f"   - **Agent Utterance:** \"{comp['agent_sentence']}\"\n"
                md_section += f"   - **Similarity Score:** {comp['similarity_score']:.4f}\n\n"
                md_section += "   -------------------------------------\n\n"
            display(Markdown(md_section))
        else:
            display(Markdown(f"**Section '{section_name}' not found in the transcript.**"))
    
    # Define weights for each section
    section_weights = {
        "Greeting Section": 0.1,
        "Issue Identification Section": 0.2,
        "Troubleshooting Section": 0.2,
        "Solution Delivery Section": 0.2,
        "Resolution (Ticket Creation) Section": 0.1,
        "Upsell (AIDA) Section": 0.1,
        "Closing Section": 0.1
    }
    
    overall_adherence = compute_overall_adherence(section_scores, section_weights)
    
    # --- Resolution Evaluation ---
    resolution_promises = extract_resolution_promises(full_transcript)
    resolution_results = []
    md_resolution = "**Resolution Verification Results:**\n\n"
    if resolution_promises:
        for promise in resolution_promises:
            result = check_resolution_action(promise)
            resolution_results.append({"promise": promise, "result": result})
            # If the promise is unfulfilled (pending), apply special formatting.
            if result['status'] == "pending":
                md_resolution += f"- **Promise:** \"{promise}\"\n"
                md_resolution += f"  - **Action Checked:** {result['action']}\n"
                md_resolution += f"  - **Status:** <span style='font-size:20px; font-weight:bold; font-style:italic;'>UNFULFILLED: {result['status']}</span>\n"
                md_resolution += f"  - **Details:** <span style='font-size:20px; font-weight:bold; font-style:italic;'>{result['details']}</span>\n\n"
            else:
                md_resolution += f"- **Promise:** \"{promise}\"\n"
                md_resolution += f"  - **Action Checked:** {result['action']}\n"
                md_resolution += f"  - **Status:** {result['status']}\n"
                md_resolution += f"  - **Details:** {result['details']}\n\n"
    else:
        md_resolution += "No resolution promises detected in this transcript.\n"
    
    display(Markdown(md_resolution))
    
    # Build the detailed feedback report, starting with overall adherence at the very beginning
    md_report = f"**Overall Script Adherence (Weighted): {overall_adherence:.2f}%**\n\n"
    md_report += "— Detailed Feedback Report —\n\n"
    md_report += "**Overall Feedback:**\n\n" + generate_overall_feedback(overall_adherence, csat_score, benchmarks["Overall"], benchmarks["CSAT"]) + "\n\n"
    md_report += "**Section-wise Feedback:**\n\n"
    detailed_fb = generate_detailed_feedback(section_scores, overall_adherence, csat_score, benchmarks)
    for section, feedback in detailed_fb["Sections"].items():
        md_report += f"- **{section}:**\n{feedback}\n\n"
    display(Markdown(md_report))
    
    # Generate qualitative coaching feedback via GPT-4 (including golden script context and resolution results)
    coaching_feedback = get_gpt4_coaching(full_transcript, section_scores, csat_score, golden_script_md, resolution_results, openai_api_key=OPENAI_API_KEY)
    display(Markdown(f"**AI-Powered Coaching Feedback:**\n\n{coaching_feedback}"))
    display(Markdown(f"**Developed by Javier Castro, dnAI**"))

if __name__ == "__main__":
    main()




**--- Greeting Section ---**

- **Adherence:** 79.77%

   - **Golden Sentence:** "Hello, thank you for calling ConnectPlus. My name is [agent name]. How may I assist you today? Good morning, and welcome to ConnectPlus support, where we’re dedicated to resolving your issues efficiently. I am [agent name], and I’m here to ensure you have a seamless experience. Whether you need technical support or general assistance, I’m here to help with any inquiries you might have."
   - **Agent Utterance:** "Good afternoon! You’ve reached the support line for ConnectPlus. My name’s Monica. How may I help you today?"
   - **Similarity Score:** 0.7700

   -------------------------------------

   - **Golden Sentence:** "Hi, thank you for reaching ConnectPlus. My name is [agent name]. What can I do for you today? Welcome to ConnectPlus customer service – I’m [agent name] and I look forward to helping you get connected and solving any issues you may face."
   - **Agent Utterance:** "Good afternoon! You’ve reached the support line for ConnectPlus. My name’s Monica. How may I help you today?"
   - **Similarity Score:** 0.8254

   -------------------------------------



**--- Issue Identification Section ---**

- **Adherence:** 47.83%

   - **Golden Sentence:** "I’m sorry to hear you are experiencing difficulties with your service. Could you please describe in detail the issues you’re facing? For instance, are you having trouble connecting or is your connection unstable? I would like to verify your account details to understand the root cause better. Your input is invaluable in helping me address the issue promptly."
   - **Agent Utterance:** "Oh no, that must be frustrating. Let’s get it sorted. Can you describe what exactly is happening?"
   - **Similarity Score:** 0.4581

   -------------------------------------

   - **Golden Sentence:** "I understand you’re encountering problems with your connection. Please explain what you’re experiencing and any error messages you might have seen. Let me also check your account and recent activity so we can pinpoint the problem."
   - **Agent Utterance:** "Oh no, that must be frustrating. Let’s get it sorted. Can you describe what exactly is happening?"
   - **Similarity Score:** 0.4984

   -------------------------------------



**--- Troubleshooting Section ---**

- **Adherence:** 68.01%

   - **Golden Sentence:** "Based on our initial check, it appears there is a disruption affecting your area. Let’s perform a few diagnostic tests to pinpoint the issue. Please check if your modem’s indicator lights are steady or blinking; this will help determine whether the signal is stable. We’ll then proceed with some troubleshooting steps such as rebooting your router if necessary."
   - **Agent Utterance:** "Okay, that confirms it. The signal isn’t stable. Just to double-check, can you reboot the router for me?"
   - **Similarity Score:** 0.7390

   -------------------------------------

   - **Golden Sentence:** "Our system shows some instability in your neighborhood. To isolate the problem, kindly confirm whether the lights on your modem are flashing or remain constant. We will try a few troubleshooting measures together, and your cooperation is much appreciated."
   - **Agent Utterance:** "It appears so. But to be sure, let’s do a quick check. Can you confirm whether your modem’s Internet light is blinking or steady?"
   - **Similarity Score:** 0.6213

   -------------------------------------



**--- Solution Delivery Section ---**

- **Adherence:** 24.13%

   - **Golden Sentence:** "Thank you for your patience. Our analysis indicates that the issue is due to a local network disruption. Our technical team has been alerted and is actively working on restoring normal service. We expect the problem to be resolved within the next two hours, and I will ensure you receive timely updates via text."
   - **Agent Utterance:** "The engineers estimate about 90 minutes. We’ll notify you via text once everything’s up and running."
   - **Similarity Score:** 0.2243

   -------------------------------------

   - **Golden Sentence:** "I appreciate your cooperation. It appears that a network disruption is affecting your service, and our engineers are addressing it as a priority. We anticipate the issue will be resolved shortly, and you’ll be notified once service is restored."
   - **Agent Utterance:** "The engineers estimate about 90 minutes. We’ll notify you via text once everything’s up and running."
   - **Similarity Score:** 0.2583

   -------------------------------------



**--- Resolution (Ticket Creation) Section ---**

- **Adherence:** 80.12%

   - **Golden Sentence:** "I understand your issue requires further investigation. Let me open a support ticket for your case right away, and I will send you a confirmation email shortly."
   - **Agent Utterance:** "I understand your issue requires further investigation. Let me open a support ticket for your case right away, and I will send you a confirmation email shortly."
   - **Similarity Score:** 1.0000

   -------------------------------------

   - **Golden Sentence:** "Your issue has been logged in our system. A support ticket has been created, and you will receive an email confirmation within a few minutes."
   - **Agent Utterance:** "I understand your issue requires further investigation. Let me open a support ticket for your case right away, and I will send you a confirmation email shortly."
   - **Similarity Score:** 0.6025

   -------------------------------------



...

Simulating API call: Checking system logs for instability.
Simulating API call: Checking for other types of resolution actions.
Simulating API call: Checking if a support ticket was created.
Simulating API call: Checking if a support ticket was created.
Simulating API call: Checking if a confirmation email was sent.


**Resolution Verification Results:**

- **Promise:** "Agent: I’m checking our systems now. There’s some instability reported in your neighborhood since early morning."
  - **Action Checked:** system check
  - **Status:** validated
  - **Details:** Instability in the area confirmed in system logs.

- **Promise:** "Agent: It appears so. But to be sure, let’s do a quick check. Can you confirm whether your modem’s Internet light is blinking or steady?"
  - **Action Checked:** other
  - **Status:** unknown
  - **Details:** No matching log found.

- **Promise:** "Agent: I understand your issue requires further investigation. Let me open a support ticket for your case right away."
  - **Action Checked:** support ticket creation
  - **Status:** validated
  - **Details:** Support ticket #T123 processed at 15:05.

- **Promise:** "Agent: I understand your issue requires further investigation. Let me open a support ticket for your case right away, and I will send you a confirmation email shortly."
  - **Action Checked:** support ticket creation
  - **Status:** validated
  - **Details:** Support ticket #T123 processed at 15:05.

- **Promise:** "Agent: Perfect! I’ve sent it to the address we have on file. You can upgrade in just one click if you’re interested."
  - **Action Checked:** email confirmation
  - **Status:** <span style='font-size:20px; font-weight:bold; font-style:italic;'>UNFULFILLED: pending</span>
  - **Details:** <span style='font-size:20px; font-weight:bold; font-style:italic;'>Latest email was not sent.</span>



**Overall Script Adherence (Weighted): 57.83%**

— Detailed Feedback Report —

**Overall Feedback:**

- **Overall Script Adherence** is 57.83% (benchmark: 75.00%). You are 17.17 points below the target.
- **Customer Satisfaction (CSAT)** is 4 out of 10 (benchmark: 7.5). You are 3.50 points below the target.


**Section-wise Feedback:**

- **Greeting Section:**
  - Your adherence score is 79.77% (benchmark: 80.00%). You are 0.23 points below the target. Consider using these golden script options:
      * Example: "Hello, thank you for calling ConnectPlus. My name is [agent name]. How may I assist you today? Good morning, and welcome to ConnectPlus support, where we’re dedicated to resolving your issues efficiently. I am [agent name], and I’m here to ensure you have a seamless experience. Whether you need technical support or general assistance, I’m here to help with any inquiries you might have."
      * Or: "Hi, thank you for reaching ConnectPlus. My name is [agent name]. What can I do for you today? Welcome to ConnectPlus customer service – I’m [agent name] and I look forward to helping you get connected and solving any issues you may face."

- **Issue Identification Section:**
  - Your adherence score is 47.83% (benchmark: 70.00%). You are 22.17 points below the target. Consider using these golden script options:
      * Example: "I’m sorry to hear you are experiencing difficulties with your service. Could you please describe in detail the issues you’re facing? For instance, are you having trouble connecting or is your connection unstable? I would like to verify your account details to understand the root cause better. Your input is invaluable in helping me address the issue promptly."
      * Or: "I understand you’re encountering problems with your connection. Please explain what you’re experiencing and any error messages you might have seen. Let me also check your account and recent activity so we can pinpoint the problem."

- **Troubleshooting Section:**
  - Your adherence score is 68.01% (benchmark: 65.00%). You are 3.01 points above the target. Great job!

- **Solution Delivery Section:**
  - Your adherence score is 24.13% (benchmark: 60.00%). You are 35.87 points below the target. Consider using these golden script options:
      * Example: "Thank you for your patience. Our analysis indicates that the issue is due to a local network disruption. Our technical team has been alerted and is actively working on restoring normal service. We expect the problem to be resolved within the next two hours, and I will ensure you receive timely updates via text."
      * Or: "I appreciate your cooperation. It appears that a network disruption is affecting your service, and our engineers are addressing it as a priority. We anticipate the issue will be resolved shortly, and you’ll be notified once service is restored."

- **Resolution (Ticket Creation) Section:**
  - Your adherence score is 80.12% (benchmark: 70.00%). You are 10.12 points above the target. Great job!

- **Upsell (AIDA) Section:**
  - Your adherence score is 62.21% (benchmark: 55.00%). You are 7.21 points above the target. Great job!

- **Closing Section:**
  - Your adherence score is 76.28% (benchmark: 75.00%). You are 1.28 points above the target. Great job!



**AI-Powered Coaching Feedback:**

**Customer Sentiment Analysis:**
- Start: The customer started the call with a neutral sentiment, stating their issue directly: "Hi. My connection has been dropping all morning."
- Middle: The sentiment slightly dips towards frustration when the customer says, "So it's a local outage?" However, the agent manages to maintain a positive interaction.
- End: The customer's sentiment seems to improve towards the end of the call, as indicated by their interest in the upsell offer and their polite farewell: "Thank you", "Oh? Interesting", "You too. Bye."

**What the Agent Did Well:**
- The agent provided a friendly and professional greeting, which set a positive tone for the interaction.
- The agent was proactive in identifying the customer's issue, and they took immediate steps to troubleshoot.
- The agent successfully opened a support ticket for the customer's case and promised to send a confirmation email.
- The agent was able to upsell the premium plan to the customer and provide them with detailed information.

**Areas for Improvement:**
- The agent didn't adhere closely to the golden script, especially in the "Solution Delivery Section" and the "Issue Identification Section."
- The agent made a promise to check the customer's modem light status, but there was no follow-up on this, leaving the promise status as unknown.
- The agent could improve on setting clear expectations for the customer. For instance, the estimated resolution time was discussed, but it did not align with the golden script, which may have caused confusion.

**Suggestions for Improvement:**
- Greeting Section: The agent's greeting was familiar but didn't adhere to the script. The agent could consider using: "Hello, thank you for calling ConnectPlus. My name is [agent name]. How may I assist you today? Good morning, and welcome to ConnectPlus support..."
- Issue Identification Section: The agent should thoroughly understand the customer's problem by asking more detailed questions. For example, "Could you please describe in detail the issues you’re facing? For instance, are you having trouble connecting or is your connection unstable?"
- Solution Delivery Section: The agent should clearly and directly explain the problem and its solution. For example, "Our analysis indicates that the issue is due to a local network disruption. Our technical team has been alerted and is actively working on restoring normal service. We expect the problem to be resolved within the next two hours, and I will ensure you receive timely updates via text."
- Resolution (Ticket Creation) Section: The agent should ensure that the customer understands the next steps and what to expect. For example, "I understand your issue requires further investigation. Let me open a support ticket for your case right away, and I will send you a confirmation email shortly."
- Upsell (AIDA) Section: The agent did well in this section but could create more urgency by saying something like: "Furthermore, we have a limited-time offer on our upgraded service package that includes bonus features and faster speeds."
- Closing Section: The agent could improve the closing section by using a more formal closing, such as, "I hope I have addressed all your concerns. Thank you for contacting ConnectPlus. We value your business and are committed to your satisfaction."

The agent should ensure that all operational actions (like opening a support ticket or sending a confirmation email) are logged and checked with the customer to close the loop on the conversation. This can be done through a system like CRM or a

**Developed by Javier Castro, dnAI, vibe coding with o3-mini high.**